# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Kenisha Ravindran</p>
### <p style="text-align: right;"> &#9989; ravind20</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

git clone https://github.com/ravind20/CMSE202-s23-turnin.git

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [47]:
#Write your code here
import pandas as pd
weather = pd.read_csv("aussie_weather_data.csv")
weather.head()
weather.dropna(inplace=True)
weather.head()

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [48]:
#Write your code here
cities = weather["Location"].unique()
print(cities)

['NorfolkIsland' 'SydneyAirport' 'WaggaWagga' 'MelbourneAirport' 'Mildura'
 'Watsonia' 'Brisbane' 'Cairns' 'Townsville' 'MountGambier' 'Nuriootpa'
 'PerthAirport' 'Perth' 'AliceSprings' 'Darwin']


#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [49]:
#Write your code here
WW = weather[weather["Location"] == "WaggaWagga"]
print(WW.head())

      Year  Month  DayOfMonth    Location  MinTemp  MaxTemp  Rainfall  \
5334  2009      1           1  WaggaWagga      9.9     28.4       0.0   
5335  2009      1           2  WaggaWagga      8.9     25.3       0.0   
5336  2009      1           3  WaggaWagga     11.0     30.7       0.0   
5337  2009      1           4  WaggaWagga     14.7     35.2       0.0   
5338  2009      1           5  WaggaWagga     11.8     36.5       0.0   

      Evaporation  Sunshine  WindGustDir  ...  Humidity9am  Humidity3pm  \
5334          9.8      13.5        157.5  ...         37.0         24.0   
5335         14.8      13.7        202.5  ...         37.0         10.0   
5336          9.8      13.7         67.5  ...         43.0         18.0   
5337          9.4      12.1         90.0  ...         46.0         19.0   
5338         11.6      13.4        180.0  ...         23.0          5.0   

      Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  \
5334       1005.2       1003.5       1

#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [50]:
#Write your code here
from sklearn.model_selection import train_test_split
features = WW.drop(['RainTomorrow'], axis=1)
target = WW['RainTomorrow']

#train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.3, random_state=4)
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.3, train_size=0.7, random_state=4)

print("Training Features Shape: ", train_features.shape)
print("Training Target Shape: ", train_target.shape)
print("Testing Features Shape: ", test_features.shape)
print("Testing Target Shape: ", test_target.shape)

Training Features Shape:  (1691, 24)
Training Target Shape:  (1691,)
Testing Features Shape:  (725, 24)
Testing Target Shape:  (725,)


## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

Logistic regression, because predict whether  it will rain tommorow is given with a yes or no answer (0 or 1) and logistic regression is good when predicting the probablity of a binary outcome

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [51]:
#Write your code here
import statsmodels.api as sm


X = WW[['MinTemp', 'MaxTemp', 'Rainfall', 'Humidity3pm', 'Pressure9am', 'Temp9am']]
y = WW['RainTomorrow']


X = sm.add_constant(X)


logit_model = sm.Logit(y, X)


result = logit_model.fit()

print(result.summary())


Optimization terminated successfully.
         Current function value: 0.304807
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 2416
Model:                          Logit   Df Residuals:                     2409
Method:                           MLE   Df Model:                            6
Date:                Fri, 21 Apr 2023   Pseudo R-squ.:                  0.3465
Time:                        21:10:42   Log-Likelihood:                -736.41
converged:                       True   LL-Null:                       -1126.9
Covariance Type:            nonrobust   LLR p-value:                2.090e-165
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          83.7265     12.324      6.794      0.000      59.573     107.880
MinTemp        -0.0226    

*Record which features are most statistically significant.*

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [88]:

y_pred = result.predict(test_features)


y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]


confusion_matrix = pd.crosstab(test_target, y_pred_binary, rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)



AssertionError: arrays and names must have the same length

In [87]:
import numpy as np

y_pred_binary = np.array(y_pred_binary)

print(y_pred_binary.shape)



(2416,)


#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

*Write your answer here.*

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [79]:

rain = WW[WW['RainTomorrow'] == True]
no_rain = WW[WW['RainTomorrow'] == False]

print(f"Rainy days: {len(rain)}")
print(f"Non-rainy days: {len(no_rain)}")
frequency = 427/2416
print("frequency:", frequency)

Rainy days: 427
Non-rainy days: 1989
frequency: 0.1767384105960265


*Write down how frequently you'd be right here.*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [82]:
#Write your code here
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


param_grid = {'C': [0.01, 0.1, 1.0, 10.0],
              'gamma': [0.01, 0.1, 1.0, 10.0],
              'kernel': ['linear', 'rbf']}


svm = SVC()


grid_search = GridSearchCV(svm, param_grid, cv=5, n_jobs=-1)


grid_search.fit(test_features, test_target)

print("Best parameters:", grid_search.best_params_)


ValueError: Found input variables with inconsistent numbers of samples: [2416, 725]

*Write down your best fir hyper-parameters here.*

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [85]:

grid_search.fit(train_features, train_target)


y_pred = grid_search.best_estimator_.predict(test_features)


from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(test_target, y_pred)
print(conf_mat)


from sklearn.metrics import precision_score, recall_score, accuracy_score
precision = precision_score(test_target, y_pred)
recall = recall_score(test_target, y_pred)
accuracy = accuracy_score(test_target, y_pred)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


ValueError: could not convert string to float: 'WaggaWagga'

## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


*Write down your response here.*
predictive model performs better than a baseline model due to its ability to capture more complex relationships between the input features and the target variable. The predictive model may be able to identify patterns and trends that are not obvious from the baseline model, which can lead to improved accuracy in predictions.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


0.84, 0.62, 352
Accuracy: 0.78
Precision: 0.83
Seed: 1234


Accuracy: 0.72
Precision: 0.69
Seed: 5678


Accuracy: 0.81
Precision: 0.88
Seed: 9876

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

*Write down your response here.*
There was some variation in the precision, recall, and accuracy in my models across different random seeds and train-test splits. For example, when using the SVM model with the default hyperparameters and a train-test split of 0.3. overall, the variation in the metrics is relatively small, suggesting that the models are performing consistently across different train-test splits and random seeds.

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

*Write down your response here.*


---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University